In [102]:
from typing import List
import sys
sys.path.append('C:/Users/kupec/OneDrive/Desktop/neuroforest-main/neuroforest-master')
import numpy as np
import json
import os
import time
from tqdm import tqdm
from main import dataloader, to_vect, Coordinates, calculate_asrs
import matplotlib.pyplot as plt
import torch

In [104]:
def features_extraction(dataloader, session_type):
    trajectories = []
    for sample in tqdm(dataloader, desc = "Data extraction"):
        session = sample["uniform"]  
        if session is not None:
            player_coords = np.array([c["coord"].to_vect() for c in session.player_coords])  
            player_coords = player_coords[:, [0, 2]]  # Keep only x and z

            if player_coords.shape[0] > 29999 :
                player_coords = player_coords[:29999] #we keep a consistent shape by removing one or two points at the end

            trajectories.append(player_coords)  # trajectories is a list of arrays, which have shape (nb_points, 2)
    return trajectories

def features_computing(trajectories):   
    speeds = []
    accelerations = [] 
    angles = []
    angular_speed = []
    angular_acceleration = []
    curvature = []
    
    for trajectory in tqdm(trajectories, desc="Computing players trajectories"):
        iterator = 0
        v_instant = []
        a_instant = []
        angle_instant = []
        angular_speed_instant = []
        angular_acceleration_instant = []
        curvature_instant = []

        while iterator < len(trajectory) - 1:  
            # Calcul de la vitesse
            delta_coords = trajectory[iterator + 1] - trajectory[iterator]
            speed = delta_coords * 25
            v_instant.append(speed)
            
            # Calcul de l'accélération
            if len(v_instant) > 1:      
                delta_v = v_instant[-2] - v_instant[-1]
                a_instant.append(delta_v * 25)
            else:
                a_instant.append(np.array([0.0, 0.0])) 

            if iterator > 0 :
                # Calcul de l'angle formé par trois points
                P1 = trajectory[iterator - 1]
                P2 = trajectory[iterator]
                P3 = trajectory[iterator + 1]
        
                vec_u = P2 - P1  
                vec_v = P3 - P2  
                
                # Normes des vecteurs
                norm_u = np.linalg.norm(vec_u)
                norm_v = np.linalg.norm(vec_v)
                
                if norm_u != 0 and norm_v != 0:  # Éviter la division par zéro
                    cos_theta = np.dot(vec_u, vec_v) / (norm_u * norm_v)
                    # Pour éviter des erreurs dues aux imprécisions numériques, on limite cos_theta à l'intervalle [-1, 1]
                    cos_theta = np.clip(cos_theta, -1.0, 1.0)
                    angle = np.arccos(cos_theta)  # Angle en radians
                else:
                    angle = 0.0 
                
                angle_instant.append(angle)
            else:
                angle_instant.append(0.0)

            
            # Calcul de la vitesse angulaire
            if len(angle_instant) > 1:
                angular_velocity = angle_instant[-1] - angle_instant[-2]
                angular_speed_instant.append(angular_velocity * 25)
            else:
                angular_speed_instant.append(0.0)  
            
            # Calcul de l'accélération angulaire
            if len(angular_speed_instant) > 1:
                angular_acceleration = angular_speed_instant[-1] - angular_speed_instant[-2]
                angular_acceleration_instant.append(angular_acceleration * 25)
            else:
                angular_acceleration_instant.append(0.0)  # Première valeur, pas d'accélération angulaire
            
            # Calcul de la courbure instantanée
            if iterator >= 1:  # On a besoin de deux points pour calculer la courbure
                x0, y0 = trajectory[iterator - 1]
                x1, y1 = trajectory[iterator]
                x2, y2 = trajectory[iterator + 1]
                # Calcul de la courbure
                numerator = (x1 - x0) * (y2 - y1) - (x2 - x1) * (y1 - y0)
                denominator = np.sqrt(((x1 - x0)**2 + (y1 - y0)**2) * ((x2 - x1)**2 + (y2 - y1)**2))
                curvature_instant.append(numerator / denominator if denominator != 0 else 0.0)
            else:
                curvature_instant.append(0.0)  # Première valeur, pas de courbure
            
            iterator += 1
        
        # Gérer les valeurs initiales pour les listes
        a_instant[0] = a_instant[1] if len(a_instant) > 1 else np.array([0.0, 0.0]) 
        angle_instant[0] = angle_instant[1] if len(angle_instant) > 1 else np.array([0.0, 0.0]) 
        angular_speed_instant[0] = angular_speed_instant[1] if len(angular_speed_instant) > 1 else np.array([0.0, 0.0]) 
        angular_acceleration_instant[0] = angular_acceleration_instant[1] if len(angular_acceleration_instant) > 1 else np.array([0.0, 0.0]) 

        speeds.append(v_instant)
        accelerations.append(a_instant)
        angles.append(angle_instant)
        angular_speed.append(angular_speed_instant)
        angular_acceleration.append(angular_acceleration_instant)
        curvature.append(curvature_instant)

    return [speeds, accelerations, angles, angular_speed, angular_acceleration, curvature]

            
def feature_reduction(feature, points_per_second, original_points_per_second = 25):
    interval = original_points_per_second // points_per_second
    reduced_feature = []
    for j in tqdm(range(len(feature))):
        reduced_feature.append([])
        for i in tqdm(range(0, len(feature[j]), interval)):
            block = feature[j][i:i + interval]
            mean_feature = np.mean(block, axis=0)
            reduced_feature[j].append(mean_feature)
    
    
    return reduced_feature

    


In [50]:
trajectories = features_extraction(dataloader, "uniform")

Data extraction: 100%|██████████| 39/39 [00:17<00:00,  2.24it/s]


In [66]:
features = features_computing(trajectories)

Computing players trajectories: 100%|██████████| 39/39 [00:07<00:00,  4.89it/s]


In [99]:
len(features[0][0])

29998

In [94]:
reduced_features = feature_reduction(features[0], 1)

100%|██████████| 39/39 [00:01<00:00, 20.23it/s]


In [98]:
len(reduced_features[0])

1200